In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [13]:
path_prefix = '.././cs5228-2310-final-project/'
df_train = pd.read_csv(path_prefix + 'train.csv')

In [14]:
df_train.head()

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,furnished,lease_commence_date,latitude,longitude,elevation,subzone,planning_area,region,monthly_rent
0,2021-09,jurong east,257,Jurong East Street 24,3 room,new generation,67.0,yes,1983,1.344518,103.738630,0.0,yuhua east,jurong east,west region,1600
1,2022-05,bedok,119,bedok north road,4-room,new generation,92.0,yes,1978,1.330186,103.938717,0.0,bedok north,bedok,east region,2250
2,2022-10,toa payoh,157,lorong 1 toa payoh,3-room,improved,67.0,yes,1971,1.332242,103.845643,0.0,toa payoh central,toa payoh,central region,1900
3,2021-08,pasir ris,250,Pasir Ris Street 21,executive,apartment,149.0,yes,1993,1.370239,103.962894,0.0,pasir ris drive,pasir ris,east region,2850
4,2022-11,kallang/whampoa,34,Whampoa West,3-room,improved,68.0,yes,1972,1.320502,103.863341,0.0,bendemeer,kallang,central region,2100


In [15]:
import math
# calculate distance by transfering latitude and longitude
def haversine_distance(lat1, lon1, lat2, lon2):
        # Radius of the Earth in kilometers
    earth_radius = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in kilometers
    distance = earth_radius * c

    return distance


In [16]:
def from_lat_long_to_dis(latA, lonA, latB, lonB):
    ra = 6378140
    rb = 6356755
    flatten = (ra - rb) / ra

    rad_lat_A = np.radians(latA)
    rad_lng_A = np.radians(lonA)
    rad_lat_B = np.radians(latB)
    rad_lng_B = np.radians(lonB)

    pA = np.arctan(rb / ra * np.tan(rad_lat_A))
    pB = np.arctan(rb / ra * np.tan(rad_lat_B))
    x = np.arccos(np.sin(pA) * np.sin(pB) + np.cos(pA) * np.cos(pB) * np.cos(rad_lng_A - rad_lng_B))
    c1 = (np.sin(x) - x) * (np.sin(pA) + np.sin(pB)) ** 2 / np.cos(x / 2) ** 2
    c2 = (np.sin(x) + x) * (np.sin(pA) - np.sin(pB)) ** 2 / np.sin(x / 2) ** 2
    dr = flatten / 8 * (c1 - c2)
    distance = ra * (x + dr)
    distance /= 1000 # convert to km
    return distance




In [17]:
#add nearest mrt distance and nearest primary school distance
def add_distance(df):
    df_mrt_existing = pd.read_csv(path_prefix + 'auxiliary-data/auxiliary-data/sg-mrt-existing-stations.csv')
    df_mrt_planned = pd.read_csv(path_prefix + 'auxiliary-data/auxiliary-data/sg-mrt-planned-stations.csv')
    df_mrt_existing['latitude'] = df_mrt_existing['latitude'].astype('float64')
    df_mrt_existing['longitude'] = df_mrt_existing['longitude'].astype('float64')
    df_mrt_planned['latitude'] = df_mrt_planned['latitude'].astype('float64')
    df_mrt_planned['longitude'] = df_mrt_planned['longitude'].astype('float64')

    for code in df_mrt_existing['code']:
        df[code] =1/from_lat_long_to_dis(df_mrt_existing[df_mrt_existing['code'] == code]['latitude'].values[0], df_mrt_existing[df_mrt_existing['code'] == code]['longitude'].values[0], df['latitude'], df['longitude']) 
    #df['NS1'] = df.apply(lambda x:haversine_distance(df_mrt_existing[df_mrt_existing['code'] == 'NS1']['latitude'], df_mrt_existing[df_mrt_existing['code'] == 'NS1']['longitude'], x['latitude'], x['longitude']) , axis=1)
    #for code in df_mrt_existing['code']:
    #df[code] = df.apply(lambda x:haversine_distance(df_mrt_existing.loc[df_mrt_existing['code'] == code]['latitude'], df_mrt_existing.loc[df_mrt_existing['code'] == code]['longitude'], x['latitude'], x['longitude']) , axis=1)
    #df['nearest_mrt'] = df.apply(lambda x: df_mrt_existing.apply(lambda y: haversine_distance(y['latitude'], y['longitude'], x['latitude'], x['longitude']) , axis=1).min(), axis=1)
    #df['nearest_primary'] = df.apply(lambda x: df_mrt_planned.apply(lambda y: haversine_distance(y['latitude'], y['longitude'], x['latitude'], x['longitude']), axis=1).min(), axis=1)
    
    return df


In [18]:
df_train = add_distance(df_train)

/tmp/ipykernel_2989/3812263480.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[code] =1/from_lat_long_to_dis(df_mrt_existing[df_mrt_existing['code'] == code]['latitude'].values[0], df_mrt_existing[df_mrt_existing['code'] == code]['longitude'].values[0], df['latitude'], df['longitude'])
/tmp/ipykernel_2989/3812263480.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[code] =1/from_lat_long_to_dis(df_mrt_existing[df_mrt_existing['code'] == code]['latitude'].values[0], df_mrt_existing[df_mrt_existing['code'] == code]['

## Data Preprocessing

In [19]:
#data clean
df_train.isnull().sum()

rent_approval_date    0
town                  0
block                 0
street_name           0
flat_type             0
                     ..
TE17                  0
TE18                  0
TE19                  0
TE20                  0
TE22                  0
Length: 178, dtype: int64

In [20]:
df_train.describe()

,floor_area_sqm,lease_commence_date,latitude,longitude,elevation,monthly_rent,NS1,NS2,NS3,NS4,...,TE12,TE13,TE14,TE15,TE16,TE17,TE18,TE19,TE20,TE22
count,60000.000000,60000.000000,60000.000000,60000.000000,60000.0,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,...,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000,60000.000000
mean,94.480458,1990.876367,1.359443,103.840110,0.0,2590.328333,0.138612,0.169804,0.155564,0.137179,...,0.126911,0.128499,0.127918,0.132626,0.148377,0.126559,0.121796,0.108289,0.103271,0.099270
std,24.082642,12.141435,0.042505,0.071627,0.0,714.910468,0.194473,0.353311,0.297576,0.256863,...,0.093314,0.103568,0.094748,0.147748,0.361526,0.192508,0.181451,0.095537,0.074745,0.055023
min,34.000000,1966.000000,1.270380,103.685228,0.0,300.000000,0.039953,0.041613,0.042170,0.040564,...,0.056567,0.057484,0.057822,0.054573,0.052971,0.050016,0.049679,0.048289,0.047503,0.046504
25%,73.000000,1981.000000,1.330939,103.778803,0.0,2100.000000,0.055436,0.058854,0.060327,0.057876,...,0.072164,0.071246,0.072027,0.069129,0.067238,0.064892,0.065362,0.064612,0.063774,0.062154
50%,93.000000,1988.000000,1.354024,103.845301,0.0,2400.000000,0.074503,0.081648,0.083650,0.077983,...,0.089687,0.087660,0.088747,0.084071,0.081446,0.077642,0.077883,0.076887,0.077054,0.081485
75%,110.000000,2000.000000,1.386968,103.897418,0.0,3000.000000,0.137859,0.143859,0.133736,0.120917,...,0.135502,0.135442,0.137350,0.125350,0.120448,0.111952,0.111114,0.108440,0.108898,0.113950
max,215.000000,2019.000000,1.457071,103.964915,0.0,6950.000000,2.786142,6.883635,12.843832,5.810454,...,0.689492,0.775769,0.730070,2.603414,19.686938,3.520160,4.802981,1.154170,0.771560,0.432611


In [21]:
#draw corr heatmap
corr = df_train.loc[:, 'monthly_rent':].corr()
#sns.heatmap(corr, annot=True, cmap='Blues')
plt.figure(figsize=(100, 100))
fig = sns.heatmap(corr, annot=True, fmt='.2f', cmap='Blues')
fig = fig.get_figure()
fig.savefig('mrtCorr.pdf')


In [24]:
print(corr['monthly_rent']>0.1)

monthly_rent    1.000000
NS1             0.011136
NS2            -0.017743
NS3            -0.033945
NS4            -0.023223
                  ...   
TE17            0.159732
TE18            0.139782
TE19            0.151424
TE20            0.145311
TE22            0.109950
Name: monthly_rent, Length: 163, dtype: float64


In [22]:
#draw corr distribution plot
#sns.pairplot(df_train.loc[:, 'monthly_rent':], kind='reg', diag_kind='kde')
